In [1]:
import os
import pandas as pd
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from dotenv import load_dotenv

In [2]:
# Set Alpaca API key and secret
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [3]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [4]:
timeframe ='1D'

In [5]:
start_date = pd.Timestamp("2010-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-10-29", tz="America/New_York").isoformat()

In [14]:
tickers = input("Enter the stock to evaluate: ") 
print(f"You entered {tickers}. Let's evaluate it...") 

Enter the stock to evaluate:  APPL


You entered APPL. Let's evaluate it...


In [7]:
# tickers = ["AAPL"]

In [8]:
df_ticker = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

In [9]:
df_ticker.head()

FB                                  
                            open   high    low    close     volume
2012-05-18 00:00:00-04:00  42.05  45.00  38.00  38.1820  478225344
2012-05-21 00:00:00-04:00  36.53  36.66  33.00  34.0255  155558610
2012-05-22 00:00:00-04:00  32.66  33.59  30.94  31.1100   94691008
2012-05-23 00:00:00-04:00  31.37  32.50  31.36  32.0000   66473588
2012-05-24 00:00:00-04:00  32.98  33.21  31.77  33.0200   43645197

In [10]:
MC_even_dist = MCSimulation(
    portfolio_data = df_ticker,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252*5
)

In [11]:
MC_even_dist.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.978345,0.974600,1.058076,0.986088,1.053944,0.993001,0.982597,1.017890,0.977828,1.005412,...,1.030485,0.992456,0.986492,1.011651,1.040225,0.981369,1.002049,1.062652,1.021152,1.044346
2,0.979450,0.985526,1.055381,0.952354,1.073917,0.987452,0.986761,0.998887,0.956477,0.976161,...,1.044311,1.050909,0.988771,1.023036,1.045124,0.972562,1.030141,1.086539,1.003356,1.017493
3,0.941330,0.969311,1.078427,0.977702,1.096584,0.965458,1.028745,1.063414,0.952385,0.978403,...,1.079626,1.051329,1.006848,1.022940,1.081199,0.934488,1.026648,1.041536,1.035351,1.030152
4,0.943414,0.951039,1.077538,0.929249,1.100051,0.970841,1.032719,1.031973,0.974142,0.973417,...,1.052051,1.119320,1.007745,1.010100,1.105904,0.919377,1.049890,1.039925,1.037926,1.041981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,0.642218,0.673921,3.260231,1.438396,12.632546,0.806806,3.969318,0.851247,2.312475,4.044045,...,1.190222,18.195570,2.959448,1.838208,2.314498,4.160776,0.840356,9.228529,1.468369,4.557360
1257,0.643610,0.675711,3.407621,1.514462,12.968572,0.772569,3.981975,0.819926,2.307228,3.923074,...,1.170003,19.563583,2.928461,1.786892,2.287263,4.190497,0.821698,8.961957,1.403561,4.667105
1258,0.625834,0.676803,3.487801,1.525104,12.546411,0.781413,4.006182,0.806964,2.329412,3.773458,...,1.158112,19.851564,3.017440,1.779330,2.293707,3.903493,0.869005,9.029476,1.409912,4.571470
1259,0.630344,0.694167,3.465876,1.500149,12.440198,0.762460,3.887037,0.788880,2.397117,3.849480,...,1.151503,20.006576,3.152103,1.792845,2.307698,3.830528,0.859463,8.983730,1.476847,4.610799


In [12]:
even_tbl = MC_even_dist.summarize_cumulative_return()
print(even_tbl)

count           1000.000000
mean               4.808947
std                4.867444
min                0.281869
25%                1.805022
50%                3.347688
75%                5.859007
max               44.615041
95% CI Lower       0.677048
95% CI Upper      18.044559
Name: 1260, dtype: float64
